# Numba/TPY

In [7]:
""" This module provides the interior differential equations for the numerical shooting method of solving the
viscoelastic-gravitational problem within a planet.

The functions here wrap those found in the radial_derivatives_dynamic.py and radial_derivatives_static.py modules to
allow for a coarser-than-required interior model to be used. For example: the user might provide a planet's interior
with radius = np.linspace(0., 6.e6, 10) with shear_modulus and bulk_modulus defined at those steps. However, the
radial solution integrator may need more steps, or steps inbetween the ones provided.

These functions use a linear interpolation to fill in the missing steps.

"""
import numpy as np
from TidalPy.constants import G, pi
from numba import njit


from TidalPy.constants import G
from TidalPy.utilities.performance import njit

@njit
def radial_derivatives_solid_general(
    radius: float,
    radial_functions: np.ndarray,
    shear_modulus, bulk_modulus, density: float,
    gravity: float, frequency: float,
    order_l: int = 2, G_to_use: float = G
    ) -> np.ndarray:
    """ Calculates the radial derivative of the radial functions in the most general form - for solid layers.

    Allows for compressibility and dynamic tides.
    Tidal harmonic l is allowed to be an integer >= 2.

    OPT: This and the other radial derivative functions are called many times during integration.
        As of TidalPy V0.3.4 this function call for floats is around 2.3 micro seconds.

    References
    ----------
    KMN15; B15; TS72

    Parameters
    ----------
    radius : FloatArray
        Radius where the radial functions are calculated. [m]
    radial_functions : np.ndarray
        Tuple of radial functions for a solid layer broken up into real and imaginary portions.
        (y1_real, y1_imag, y2_real, y2_imag, y3_real, y3_imag, y4_real, y4_imag, y5_real, y5_imag, y6_real, y6_imag)
    shear_modulus : Numeric
        Shear modulus (can be complex for dissipation) at `radius` [Pa]
    bulk_modulus : Numeric
        Bulk modulus (can be complex for dissipation) at `radius` [Pa]
    density : FloatArray
        Density at `radius` [kg m-3]
    gravity : FloatArray
        Acceleration due to gravity at `radius` [m s-2]
    frequency : FloatArray
        Forcing frequency (for spin-synchronous tides this is the orbital motion) [rad s-1]
    order_l : int = 2
        Tidal harmonic order.
    G_to_use : float = G
        Newton's gravitational constant. This can be provided as in its MKS units or dimensionless to match the other
        inputs.

    Returns
    -------
    radial_derivatives : np.ndarray
        The derivatives of the radial functions for a solid layer (dynamic assumption)

    """

    y1_real = radial_functions[0]
    y1_imag = radial_functions[1]
    y2_real = radial_functions[2]
    y2_imag = radial_functions[3]
    y3_real = radial_functions[4]
    y3_imag = radial_functions[5]
    y4_real = radial_functions[6]
    y4_imag = radial_functions[7]
    y5_real = radial_functions[8]
    y5_imag = radial_functions[9]
    y6_real = radial_functions[10]
    y6_imag = radial_functions[11]

    # Convert floats to complex
    y1 = y1_real + 1.0j * y1_imag
    y2 = y2_real + 1.0j * y2_imag
    y3 = y3_real + 1.0j * y3_imag
    y4 = y4_real + 1.0j * y4_imag
    y5 = y5_real + 1.0j * y5_imag
    y6 = y6_real + 1.0j * y6_imag

    # Convert compressibility parameters (the first lame parameter can be complex)
    lame = (bulk_modulus - (2. / 3.) * shear_modulus)

    # Optimizations
    lp1              = order_l + 1.
    lm1              = order_l - 1.
    llp1             = order_l * lp1
    lame_2mu         = lame + 2. * shear_modulus
    lame_2mu_inverse = 1. / lame_2mu
    r_inverse        = 1. / radius
    two_shear_r_inv  = 2. * shear_modulus * r_inverse
    density_gravity  = density * gravity
    dynamic_term     = -frequency * frequency * density * radius
    grav_term        = 4. * pi * G_to_use * density
    y1_y3_term       = 2. * y1 - llp1 * y3

    # See Eq. 82 in TS72 or Eqs. 4--9 in KMN15 or Eqs. 13--18 in B15
    #   Note: There appears to be a missing factor of mu^2 in some of the terms in KMN15.
    # dy2 and dy4 contain all three of: dynamic, viscoelastic, and gravitational terms.
    dy1 = lame_2mu_inverse * (
            y1_y3_term * -lame * r_inverse +
            y2
    )

    dy2 = r_inverse * (
            y1 * (dynamic_term - 2. * density_gravity) +
            y2 * -2. +
            y4 * llp1 +
            y5 * density * lp1 +
            y6 * -density * radius +
            dy1 * 2. * lame +
            y1_y3_term * (2. * (lame + shear_modulus) * r_inverse - density_gravity)
    )

    dy3 = \
        y1 * -r_inverse + \
        y3 * r_inverse + \
        y4 * (1. / shear_modulus)

    dy4 = r_inverse * (
            y1 * (density_gravity + two_shear_r_inv) +
            y3 * (dynamic_term - two_shear_r_inv) +
            y4 * -3. +
            y5 * -density +
            dy1 * -lame +
            y1_y3_term * -lame_2mu * r_inverse
    )

    dy5 = \
        y1 * grav_term + \
        y5 * -lp1 * r_inverse + \
        y6

    dy6 = r_inverse * (
            y1 * grav_term * lm1 +
            y6 * lm1 +
            y1_y3_term * grav_term
    )

    # Build output
    dy = np.empty(12, dtype=np.float64)

    # Convert back to floats
    dy[0] = np.real(dy1)
    dy[1] = np.imag(dy1)
    dy[2] = np.real(dy2)
    dy[3] = np.imag(dy2)
    dy[4] = np.real(dy3)
    dy[5] = np.imag(dy3)
    dy[6] = np.real(dy4)
    dy[7] = np.imag(dy4)
    dy[8] = np.real(dy5)
    dy[9] = np.imag(dy5)
    dy[10] = np.real(dy6)
    dy[11] = np.imag(dy6)

    return dy

@njit
def dynamic_solid_ode(
    radius: float,
    y_vector: np.ndarray,
    radius_array: np.ndarray, shear_modulus_array: np.ndarray, bulk_modulus_array: np.ndarray,
    density_array: np.ndarray, gravity_array: np.ndarray, frequency: float,
    order_l: int = 2, G_to_use: float = G, incompressible: bool = False
    ) -> np.ndarray:
    """ A njit-safe radial derivative function for static, solid layers.

    Parameters
    ----------
    radius : float
        Requested radius to at which the radial derivatives are calculated [m]
    y_vector : np.ndarray
        The radial functions at (or near) the provided radius. Used to estimate the derivatives.
    radius_array : np.ndarray
        Array of radius_array for the interior of a planet or layer where this radial derivative function is valid.
        The bottom most radius_array of a planet should not be equal to zero [m]
    shear_modulus_array : np.ndarray
        Shear modulus at each `radius_array` [Pa] (can be complex for shear dissipation)
    bulk_modulus_array : np.ndarray
        Bulk modulus at each `radius_array` [Pa] (can be complex for bulk dissipation)
    density_array : np.ndarray
        Density at each `radius_array` [kg m-3]
    gravity_array : np.ndarray
        Acceleration due to gravity calculated at each `radius_array` [m s-2]
    frequency : float
        Forcing frequency [rad s-1]
    order_l : int = 2
        Tidal harmonic order
    G_to_use : float = G
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    incompressible : bool = False
        If `True`, the incompressible assumption will be used.

    Returns
    -------
    solid_dyn_derivatives : np.ndarray
        The radial derivatives estimated using the provided radius for a solid layer under the dynamic assumption.
    """

    # These physical parameters are generally given in a coarser resolution than what is required during integration.
    #    Therefore, we interpolate their values at the integrator's requested `radius`.
    shear_modulus = np.interp(radius, radius_array, shear_modulus_array)
    density = np.interp(radius, radius_array, density_array)
    gravity = np.interp(radius, radius_array, gravity_array)

    bulk_modulus = np.interp(radius, radius_array, bulk_modulus_array)
    solid_dyn_derivatives = \
        radial_derivatives_solid_general(
            radius, y_vector, shear_modulus, bulk_modulus, density, gravity, frequency,
            order_l=order_l, G_to_use=G_to_use
            )

    return solid_dyn_derivatives

# Cython

In [1]:
%load_ext Cython

In [27]:
%%cython --annotate --force
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True
""" Functions to calculate the Rayleigh wave propagation differential equations

These functions are the most general and allow for:
    - Compressibility
    - Dynamic (non-static) Tides
    - Liquid layer propagation

References
----------
KMN15 : Kamata+ (2015; JGR-P; DOI: 10.1002/2015JE004821)
B15   : Beuthe+ (2015; Icarus; DOI: 10.1016/j.icarus.2015.06.008)
TMS05 : Tobie+ (2005; Icarus; DOI: 10.1016/j.icarus.2005.04.006)
S74   : Saito (1974; J. Phy. Earth; DOI: 10.4294/jpe1952.22.123)
TS72  : Takeuchi, H., and M. Saito (1972), Seismic surface waves, Methods Comput. Phys., 11, 217–295.
"""
import cython
import numpy as np
cimport numpy as np
np.import_array()

from cython.parallel import prange

from libc.math cimport isnan, M_PI

# Get machine precision.
cdef double EPS
EPS = np.finfo(dtype=np.float64).eps

cdef double G = 6.67430e-11

# Determine cache limits.
cdef int LIKELY_IN_CACHE_SIZE = 8

@cython.exceptval(check=False)
@cython.nogil
cdef int binary_search_with_guess(double key, double[:] array, int length, int guess) nogil:
    """ Binary search with guess.
    
    Based on `numpy`'s `binary_search_with_guess` function.
    
    Parameters
    ----------
    key : float
        Key index to search for.
    array : np.ndarray
        Array to search in.
    length : int
        Length of array.
    guess : int 
        Initial guess of where key might be.
    
    Returns
    -------
    guess : int
        Corrected guess after search.
    """

    cdef int imin = 0
    cdef int imax = length

    if key > array[length - 1]:
        return length
    elif key < array[0]:
        return -1

    # If len <= 4 use linear search.
#     if length <= 4:
#         raise NotImplemented

    if guess > (length - 3):
        guess = length - 3
    if guess < 1:
        guess = 1

    # check most likely values: guess - 1, guess, guess + 1
    if key < array[guess]:
        if key < array[guess - 1]:
            imax = guess - 1
            # last attempt to restrict search to items in cache
            if guess > LIKELY_IN_CACHE_SIZE and key >= array[guess - LIKELY_IN_CACHE_SIZE]:
                imin = guess - LIKELY_IN_CACHE_SIZE

        else:
            return guess - 1
    else:
        if key < array[guess + 1]:
            return guess
        else:
            if key < array[guess + 2]:
                return guess + 1
            else:
                imin = guess + 2
                # last attempt to restrict search to items in cache
                if guess < (length - LIKELY_IN_CACHE_SIZE - 1) and key < array[guess + LIKELY_IN_CACHE_SIZE]:
                    imax = guess + LIKELY_IN_CACHE_SIZE

    # Finally, find index by bisection
    cdef int imid
    while imin < imax:
        imid = imin + ((imax - imin) >> 1)
        if key >= array[imid]:
            imin = imid + 1
        else:
            imax = imid

    return imin - 1

@cython.exceptval(check=False)
@cython.nogil
cdef double interp(double desired_x, double[:] x_domain, double[:] dependent_values) nogil:
    """ Interpolation function for floats.
    
    Provided a domain, `x_domain` and a dependent array `dependent_values` search domain for value closest to 
    `desired_x` and return the value of `dependent_values` at that location if it is defined. Otherwise, use local 
    slopes of `x_domain` and `dependent_values` to interpolate a value of `dependent_values` at `desired_x`.

    Based on `numpy`'s `interp` function.

    Parameters
    ----------
    desired_x : float
        Location where `dependent_variables` is desired.
    x_domain : np.ndarray[float]
        Domain to search for the correct location.
    dependent_values : np.ndarray[float]
        Dependent values that are to be returned after search and interpolation.

    Returns
    -------
    result : float
        Desired value of `dependent_values`.
    
    """

    cdef int lenx
    lenx = len(x_domain)
    # TODO: Needs to be at least 3 item long array. Add exception here?

    cdef double left_value
    left_value = dependent_values[0]
    cdef double right_value
    right_value = dependent_values[lenx - 1]

    # Binary Search with Guess
    cdef int i, j
    j = 0
    cdef double slope

    cdef double result
    cdef double fp_at_j
    cdef double xp_at_j
    cdef double fp_at_jp1
    cdef double xp_at_jp1

    # Perform binary search with guess
    j = binary_search_with_guess(desired_x, x_domain, lenx, j)

    if j == -1:
        result = left_value
    elif j == lenx:
        result = right_value
    else:
        fp_at_j = dependent_values[j]
        xp_at_j = x_domain[j]
        if j == lenx - 1:
            result = fp_at_j
        elif xp_at_j == desired_x:
            result = fp_at_j
        else:
            fp_at_jp1 = dependent_values[j + 1]
            xp_at_jp1 = x_domain[j + 1]
            slope = (fp_at_jp1 - fp_at_j) / (xp_at_jp1 - xp_at_j)

            # If we get nan in one direction, try the other
            result = slope * (desired_x - xp_at_j) + fp_at_j
            if isnan(result):
                result = slope * (desired_x - xp_at_jp1) + fp_at_jp1
                if isnan(result) and (fp_at_jp1 == fp_at_j):
                    result = fp_at_j

    return result


@cython.exceptval(check=False)
@cython.nogil
cdef double complex interp_complex(double desired_x, double[:] x_domain,
                                    double complex[:] dependent_variables) nogil:
    """ Interpolation function for complex numbers.

    Provided a domain, `desired_x` and a dependent array `dependent_values` search domain for value closest to 
    `desired_x` and return the value of `dependent_values` at that location if it is defined. Otherwise, use local 
    slopes of `desired_x` and `dependent_values` to interpolate a value of `dependent_values` at `desired_x`.

    Based on `numpy`'s `interp` function.

    Parameters
    ----------
    desired_x : float
        Location where `dependent_variables` is desired.
    desired_x : np.ndarray[float]
        Domain to search for the correct location.
    dependent_values : np.ndarray[complex]
        Dependent values that are to be returned after search and interpolation.

    Returns
    -------
    result : complex
        Desired value of `dependent_values`.

    """
    
    cdef int lenx
    lenx = len(x_domain)
    # Note: Needs to be at least 3 item long array. Add exception here?
    
    cdef double complex left_value
    left_value = dependent_variables[0]
    cdef double complex right_value
    right_value = dependent_variables[lenx - 1]
    
    # Binary Search with Guess
    cdef int i, j
    j = 0
    cdef double slope_real
    cdef double slope_imag
    cdef double x_slope_inverse
    
    cdef double result_real
    cdef double result_imag
    cdef double fp_at_j_real
    cdef double fp_at_j_imag
    cdef double xp_at_j
    cdef double fp_at_jp1_real
    cdef double fp_at_jp1_imag
    cdef double xp_at_jp1

    # Perform binary search with guess
    j = binary_search_with_guess(desired_x, x_domain, lenx, j)
    
    if j == -1:
        result_real = left_value.real
        result_imag = left_value.imag
    elif j == lenx:
        result_real = right_value.real
        result_imag = right_value.imag
    else:
        fp_at_j_real = dependent_variables[j].real
        fp_at_j_imag = dependent_variables[j].imag
        xp_at_j = x_domain[j]
        if j == lenx - 1:
            result_real = fp_at_j_real
            result_imag = fp_at_j_imag
        elif xp_at_j == desired_x:
            result_real = fp_at_j_real
            result_imag = fp_at_j_imag
        else:
            fp_at_jp1_real = dependent_variables[j + 1].real
            fp_at_jp1_imag = dependent_variables[j + 1].imag
            xp_at_jp1 = x_domain[j + 1]
            x_slope_inverse = 1.0 / (xp_at_jp1 - xp_at_j)
            slope_real = (fp_at_jp1_real - fp_at_j_real) * x_slope_inverse
            slope_imag = (fp_at_jp1_imag - fp_at_j_imag) * x_slope_inverse
    
            # If we get nan in one direction try the other
            # Real Part
            result_real = slope_real * (desired_x - xp_at_j) + fp_at_j_real
            if isnan(result_real):
                result_real = slope_real * (desired_x - xp_at_jp1) + fp_at_jp1_real
                if isnan(result_real) and (fp_at_jp1_real == fp_at_j_real):
                    result_real = fp_at_j_real
            
            # Imaginary Part
            result_imag = slope_imag * (desired_x - xp_at_j) + fp_at_j_imag
            if isnan(result_imag):
                result_imag = slope_imag * (desired_x - xp_at_jp1) + fp_at_jp1_imag
                if isnan(result_imag) and (fp_at_jp1_imag == fp_at_j_imag):
                    result_imag = fp_at_j_imag
                    
    cdef double complex result
    result = result_real + 1.0j * result_imag
                    
    return result

@cython.exceptval(check=False)
@cython.nogil
cdef void radial_derivatives_solid_general_x(
    double radius,
    double[:] radial_functions,
    double[:] dy,
    double complex shear_modulus, double bulk_modulus, double density,
    double gravity, double frequency,
    int order_l=2, double G_to_use=G
    ) nogil:
    """ Calculates the radial derivative of the radial functions in the most general form - for solid layers.

    Allows for compressibility and dynamic tides.
    Tidal harmonic l is allowed to be an integer >= 2.

    OPT: This and the other radial derivative functions are called many times during integration.
        As of TidalPy V0.3.4 this function call for floats is around 2.3 micro seconds.

    References
    ----------
    KMN15; B15; TS72

    Parameters
    ----------
    radius : FloatArray
        Radius where the radial functions are calculated. [m]
    radial_functions : np.ndarray
        Tuple of radial functions for a solid layer broken up into real and imaginary portions.
        (y1_real, y1_imag, y2_real, y2_imag, y3_real, y3_imag, y4_real, y4_imag, y5_real, y5_imag, y6_real, y6_imag)
    shear_modulus : NumArray
        Shear modulus (can be complex for dissipation) at `radius` [Pa]
    bulk_modulus : NumArray
        Bulk modulus (can be complex for dissipation) at `radius` [Pa]
    density : FloatArray
        Density at `radius` [kg m-3]
    gravity : FloatArray
        Acceleration due to gravity at `radius` [m s-2]
    frequency : FloatArray
        Forcing frequency (for spin-synchronous tides this is the orbital motion) [rad s-1]
    order_l : int = 2
        Tidal harmonic order.
    G_to_use : float = G
        Newton's gravitational constant. This can be provided as in its MKS units or dimensionless to match the other
        inputs.

    Returns
    -------
    radial_derivatives : np.ndarray
        The derivatives of the radial functions for a solid layer (dynamic assumption)

    """

    cdef double y1_real = radial_functions[0]
    cdef double y1_imag = radial_functions[1]
    cdef double y2_real = radial_functions[2]
    cdef double y2_imag = radial_functions[3]
    cdef double y3_real = radial_functions[4]
    cdef double y3_imag = radial_functions[5]
    cdef double y4_real = radial_functions[6]
    cdef double y4_imag = radial_functions[7]
    cdef double y5_real = radial_functions[8]
    cdef double y5_imag = radial_functions[9]
    cdef double y6_real = radial_functions[10]
    cdef double y6_imag = radial_functions[11]

    # Convert floats to complex
    cdef double complex y1 = y1_real + 1.0j * y1_imag
    cdef double complex y2 = y2_real + 1.0j * y2_imag
    cdef double complex y3 = y3_real + 1.0j * y3_imag
    cdef double complex y4 = y4_real + 1.0j * y4_imag
    cdef double complex y5 = y5_real + 1.0j * y5_imag
    cdef double complex y6 = y6_real + 1.0j * y6_imag

    # Convert compressibility parameters (the first lame parameter can be complex)
    cdef double complex lame = (<double complex>bulk_modulus - (2. / 3.) * shear_modulus)

    # Optimizations
    cdef double degree_l_flt = <double>order_l
    cdef double lp1              = degree_l_flt + 1.
    cdef double lm1              = degree_l_flt - 1.
    cdef double llp1             = degree_l_flt * lp1
    cdef double complex lame_2mu         = lame + 2. * shear_modulus
    cdef double complex lame_2mu_inverse = 1. / lame_2mu
    cdef double r_inverse        = 1. / radius
    cdef double complex two_shear_r_inv  = 2. * shear_modulus * r_inverse
    cdef double density_gravity  = density * gravity
    cdef double dynamic_term     = -frequency * frequency * density * radius
    cdef double grav_term        = 4. * M_PI * G_to_use * density
    cdef double complex y1_y3_term       = 2. * y1 - llp1 * y3

    # See Eq. 82 in TS72 or Eqs. 4--9 in KMN15 or Eqs. 13--18 in B15
    #   Note: There appears to be a missing factor of mu^2 in some of the terms in KMN15.
    # dy2 and dy4 contain all three of: dynamic, viscoelastic, and gravitational terms.
    cdef double complex dy1
    cdef double complex dy2
    cdef double complex dy3
    cdef double complex dy4
    cdef double complex dy5
    cdef double complex dy6

    dy1 = lame_2mu_inverse * (
            y1_y3_term * -lame * r_inverse +
            y2
    )

    dy2 = r_inverse * (
            y1 * (dynamic_term - 2. * density_gravity) +
            y2 * -2. +
            y4 * llp1 +
            y5 * density * lp1 +
            y6 * -density * radius +
            dy1 * 2. * lame +
            y1_y3_term * (2. * (lame + shear_modulus) * r_inverse - density_gravity)
    )

    dy3 = \
        y1 * -r_inverse + \
        y3 * r_inverse + \
        y4 * (1. / shear_modulus)

    dy4 = r_inverse * (
            y1 * (density_gravity + two_shear_r_inv) +
            y3 * (dynamic_term - two_shear_r_inv) +
            y4 * -3. +
            y5 * -density +
            dy1 * -lame +
            y1_y3_term * -lame_2mu * r_inverse
    )

    dy5 = \
        y1 * grav_term + \
        y5 * -lp1 * r_inverse + \
        y6

    dy6 = r_inverse * (
            y1 * grav_term * lm1 +
            y6 * lm1 +
            y1_y3_term * grav_term
    )

    # Convert back to floats
    dy[0] = dy1.real
    dy[1] = dy1.imag
    dy[2] = dy2.real
    dy[3] = dy2.imag
    dy[4] = dy3.real
    dy[5] = dy3.imag
    dy[6] = dy4.real
    dy[7] = dy4.imag
    dy[8] = dy5.real
    dy[9] = dy5.imag
    dy[10] = dy6.real
    dy[11] = dy6.imag


@cython.exceptval(check=False)
@cython.nogil
cpdef void dynamic_solid_ode_x(double radius, double[:] y_vector, double[:] dy_vector,
                             double[:] radius_array, double complex[:] shear_modulus_array,
                             double[:] bulk_modulus_array, double[:] density_array, double[:] gravity_array,
                             double frequency, int order_l = 2, double G_to_use = G):
    """ A njit-safe radial derivative function for static, solid layers.

    Parameters
    ----------
    radius : float
        Requested radius to at which the radial derivatives are calculated [m]
    y_vector : np.ndarray
        The radial functions at (or near) the provided radius. Used to estimate the derivatives.
    radius_array : np.ndarray
        Array of radius_array for the interior of a planet or layer where this radial derivative function is valid.
        The bottom most radius_array of a planet should not be equal to zero [m]
    shear_modulus_array : np.ndarray
        Shear modulus at each `radius_array` [Pa] (can be complex for shear dissipation)
    bulk_modulus_array : np.ndarray
        Bulk modulus at each `radius_array` [Pa] (can be complex for bulk dissipation)
    density_array : np.ndarray
        Density at each `radius_array` [kg m-3]
    gravity_array : np.ndarray
        Acceleration due to gravity calculated at each `radius_array` [m s-2]
    frequency : float
        Forcing frequency [rad s-1]
    order_l : int = 2
        Tidal harmonic order
    G_to_use : float = G
        Gravitational constant. Provide a non-dimensional version if the rest of the inputs are non-dimensional.
    incompressible : bool = False
        If `True`, the incompressible assumption will be used.

    Returns
    -------
    solid_dyn_derivatives : np.ndarray
        The radial derivatives estimated using the provided radius for a solid layer under the dynamic assumption.
    """

    # These physical parameters are generally given in a coarser resolution than what is required during integration.
    #    Therefore, we interpolate their values at the integrator's requested `radius`.
    cdef double complex shear_modulus = interp_complex(radius, radius_array, shear_modulus_array)
    cdef double density = interp(radius, radius_array, density_array)
    cdef double gravity = interp(radius, radius_array, gravity_array)
    cdef double bulk_modulus = interp(radius, radius_array, bulk_modulus_array)

    radial_derivatives_solid_general_x(
        radius, y_vector, dy_vector, shear_modulus, bulk_modulus, density, gravity, frequency,
        order_l, G_to_use
        )


Content of stdout:
_cython_magic_c8add54ee600812505a21d285b7efc790cff49ae.c
C:\ProgramData\Anaconda3\envs\py310\lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\joepr\.ipython\cython\_cython_magic_c8add54ee600812505a21d285b7efc790cff49ae.c(1511): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\joepr\.ipython\cython\_cython_magic_c8add54ee600812505a21d285b7efc790cff49ae.c(1462): note: see previous definition of '__pyx_nonatomic_int_type'
C:\Users\joepr\.ipython\cython\_cython_magic_c8add54ee600812505a21d285b7efc790cff49ae.c(19265): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\joepr\.ipython\cython\_cython_magic_c8add54ee600812505a21d285b7efc790cff49ae.c(19629): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
   Creating library C:\Users\joepr\.ipyth

In [13]:
radius_array = np.linspace(0.1, 1.0e6, 100)
shear_modulus_array = (1.0e10 -1.0e-1j) * np.ones(radius_array.shape, dtype=np.complex128)
bulk_modulus_array = 1.0e10 * np.ones_like(radius_array)
density_array = np.linspace(10000, 4000, 100)
gravity_array = np.linspace(0.1, 9., 100)
frequency = 1.0e-6
order_l = 2
G_to_use = G

radius = 6.5e5
y_vector = np.asarray((1.0e-4, -1.0e-4, 1.0e-4, -1.0e-4, 1.0e-4, -1.0e-4, 1.0e-4, -1.0e-4, 1.0e-4, -1.0e-4, 1.0e-4, -1.0e-4), dtype=np.float64)
dy_vector = np.empty_like(y_vector)

In [26]:
# Numba
# 3.59us 3.6us 3.59us
out = dynamic_solid_ode(radius, y_vector, radius_array, shear_modulus_array, bulk_modulus_array, density_array, gravity_array, frequency, order_l, G_to_use)
print(out)

%timeit dynamic_solid_ode(radius, y_vector, radius_array, shear_modulus_array, bulk_modulus_array, density_array, gravity_array, frequency, order_l, G_to_use)

[8.792e-11 -8.792e-11 -6.100e-01 6.100e-01 1.000e-14 -1.000e-14 2.622e-05
 -2.622e-05 1.000e-04 -1.000e-04 1.538e-10 -1.538e-10]
3.68 µs ± 77.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [25]:
# Cython
# 2.95us 2.93us 2.93us
# 2.91us

dynamic_solid_ode_x(radius, y_vector, dy_vector,
                    radius_array, shear_modulus_array,
                    bulk_modulus_array, density_array, gravity_array,
                    frequency, order_l, G_to_use)
print(dy_vector)

%timeit dynamic_solid_ode_x(radius, y_vector, dy_vector, radius_array, shear_modulus_array, bulk_modulus_array, density_array, gravity_array, frequency, order_l, G_to_use)

[8.792e-11 -8.792e-11 -6.100e-01 6.100e-01 1.000e-14 -1.000e-14 2.622e-05
 -2.622e-05 1.000e-04 -1.000e-04 1.538e-10 -1.538e-10]
2.95 µs ± 28.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [154]:
%%cython --annotate --force
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

import cython

from libc.math cimport pi

cdef double G
G = 6.67430e-11
@cython.exceptval(check=False)
cdef void dy_liquid_dynamic_incompressible_x(
    double radius, double[:] radial_functions, double[:] dy,
    double density, double gravity, double frequency,
    unsigned int degree_l=2, double G_to_use=G) nogil:
    """ Calculates the radial derivative of the radial functions - for incompressible solid layers.

    Allows for dynamic tides.
    Tidal degree l is allowed to be an integer >= 2.

    References
    ----------
    KMN15; B15; TS72

    Parameters
    ----------
    radius : float
        Radius where the radial functions are calculated. [m]
    radial_functions : np.ndarray
        Tuple of radial functions for a solid layer broken up into real and imaginary portions.
        (y1_real, y1_imag, y2_real, y2_imag, y3_real, y3_imag, y4_real, y4_imag, y5_real, y5_imag, y6_real, y6_imag)
    dy : np.ndarray
        Derivative of the radial functions with respect to radius.
    shear_modulus : float
        Shear modulus (can be complex for dissipation) at `radius` [Pa]
    density : float
        Density at `radius` [kg m-3]
    gravity : float
        Acceleration due to gravity at `radius` [m s-2]
    frequency : float
        Forcing frequency (for spin-synchronous tides this is the orbital motion) [rad s-1]
    degree_l : int = 2
        Tidal harmonic degree.
    G_to_use : float = G
        Newton's gravitational constant. This can be provided as in its MKS units or dimensionless to match the other
        inputs.

    """

    # Pull out y values
    cdef double y1_real, y2_real, y5_real, y6_real
    cdef double y1_imag, y2_imag, y5_imag, y6_imag

    y1_real = radial_functions[0]
    y1_imag = radial_functions[1]
    y2_real = radial_functions[2]
    y2_imag = radial_functions[3]
    y5_real = radial_functions[4]
    y5_imag = radial_functions[5]
    y6_real = radial_functions[6]
    y6_imag = radial_functions[7]

    # Convert floats to complex
    cdef double complex y1, y2, y5, y6

    y1 = y1_real + 1.0j * y1_imag
    y2 = y2_real + 1.0j * y2_imag
    y5 = y5_real + 1.0j * y5_imag
    y6 = y6_real + 1.0j * y6_imag

    # Optimizations
    cdef double degree_l_flt, lp1, lm1, llp1
    cdef double r_inverse, density_gravity, dynamic_term, grav_term

    degree_l_flt     = <double>degree_l
    lp1              = degree_l_flt + 1.
    lm1              = degree_l_flt - 1.
    llp1             = degree_l_flt * lp1
    r_inverse        = 1. / radius
    density_gravity  = density * gravity
    dynamic_term     = -frequency * frequency * density * radius
    grav_term        = 4. * pi * G_to_use * density
    
    # y3 derivative is undetermined for a liquid layer, but we can calculate its value which is still used in the
    #   other derivatives.
    cdef double complex y3, y1_y3_term
    y3 = (1. / dynamic_term) * (y2 + density * y5 - density_gravity * y1)
    y1_y3_term = 2. * y1 - llp1 * y3

    
    cdef double complex dy1, dy2, dy5, dy6

    dy1 = y1_y3_term * -r_inverse

    dy2 = r_inverse * (
            y1 * (dynamic_term - 2. * density_gravity) +
            y5 * density * lp1 +
            y6 * -density * radius +
            # TODO: In the solid version there is a [2. * (lame + shear_modulus) * r_inverse] coefficient for y1_y3_term
            #   In TS72 the first term is gone. Shouldn't Lame + mu = Lame = Bulk for liquid layer?
            y1_y3_term * -density_gravity
    )

    dy5 = \
        y1 * grav_term + \
        y5 * -lp1 * r_inverse + \
        y6

    dy6 = r_inverse * (
            y1 * grav_term * lm1 +
            y6 * lm1 +
            y1_y3_term * grav_term
    )

    # Convert back to floats
    dy[0] = dy1.real
    dy[1] = dy1.imag
    dy[2] = dy2.real
    dy[3] = dy2.imag
    dy[4] = dy5.real
    dy[5] = dy5.imag
    dy[6] = dy6.real
    dy[7] = dy6.imag

Content of stdout:
_cython_magic_6ed88a90e4349e343f1bb2386b32bc5bb0aeb7ed.c
C:\Users\joepr\.ipython\cython\_cython_magic_6ed88a90e4349e343f1bb2386b32bc5bb0aeb7ed.c(1488): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\joepr\.ipython\cython\_cython_magic_6ed88a90e4349e343f1bb2386b32bc5bb0aeb7ed.c(1439): note: see previous definition of '__pyx_nonatomic_int_type'
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_6ed88a90e4349e343f1bb2386b32bc5bb0aeb7ed.cp310-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_6ed88a90e4349e343f1bb2386b32bc5bb0aeb7ed.cp310-win_amd64.exp
Generating code
Finished generating code

In [172]:
from numba import njit
import numpy as np
from TidalPy.constants import G, pi

@njit
def radial_derivatives_solid_general(
    radius: float,
    radial_functions: np.ndarray,
    shear_modulus, density: float,
    gravity: float,
    order_l: int = 2, G_to_use: float = G
    ) -> np.ndarray:
    """ Calculates the derivatives of the radial functions using the static assumption - for solid layers.

    Assumes incompressible and static tides.
    Dynamic tides are not permitted (w=0)
    Tidal harmonic l is allowed to be >= 2.

    References
    ----------
    KMN15; B15; TS72

    Parameters
    ----------
    radius : float
        Radius where the radial functions are calculated. [m; or dimensionless]
    radial_functions : Numeric
        Tuple of radial functions for a solid layer (y1, y2, y3, y4, y5, y6)
    shear_modulus : NumArray
        Shear modulus (can be complex for dissipation) at `radius` [Pa; or dimensionless]
    density : float
        Density at `radius` [kg m-3; or dimensionless]
    gravity : float
        Acceleration due to gravity at `radius` [m s-2; or dimensionless]
    order_l : int = 2
        Tidal harmonic order.
    G_to_use : float = G
        Newton's gravitational constant. This can be provided as in its MKS units or dimensionless to match the other
        inputs.

    Returns
    -------
    radial_derivatives : np.ndarray
        The radial derivatives of the radial functions

    """

    y1_real = radial_functions[0]
    y1_imag = radial_functions[1]
    y2_real = radial_functions[2]
    y2_imag = radial_functions[3]
    y3_real = radial_functions[4]
    y3_imag = radial_functions[5]
    y4_real = radial_functions[6]
    y4_imag = radial_functions[7]
    y5_real = radial_functions[8]
    y5_imag = radial_functions[9]
    y6_real = radial_functions[10]
    y6_imag = radial_functions[11]

    # Convert floats to complex
    y1 = y1_real + 1.0j * y1_imag
    y2 = y2_real + 1.0j * y2_imag
    y3 = y3_real + 1.0j * y3_imag
    y4 = y4_real + 1.0j * y4_imag
    y5 = y5_real + 1.0j * y5_imag
    y6 = y6_real + 1.0j * y6_imag

    # Convert compressibility parameters (the first lame parameter can be complex)
    #lame = (bulk_modulus - (2. / 3.) * shear_modulus)

    # Optimizations
    lp1 = order_l + 1.
    lm1 = order_l - 1.
    llp1 = order_l * lp1
    #lame_2mu = lame + 2. * shear_modulus
    #lame_2mu_inverse = 1. / lame_2mu
    r_inverse = 1. / radius
    two_shear_r_inv = 2. * shear_modulus * r_inverse
    density_gravity = density * gravity
    grav_term = 4. * pi * G_to_use * density
    y1_y3_term = 2. * y1 - llp1 * y3

    # See Eq. 82 in TS72 or Eqs. 4--9 in KMN15 or Eqs. 13--18 in B15
    #   Note: There appears to be a missing factor of mu^2 in some of the terms in KMN15.
    # The static case just sets all frequency dependence in these equations to zero.
    # dy2 and dy4 contain all three of: dynamic, viscoelastic, and gravitational terms.
    dy1 = y1_y3_term * -1. * r_inverse

    dy2 = r_inverse * (
            y1 * (12. * shear_modulus * r_inverse - 4. * density_gravity) +
            y3 * llp1 * (density_gravity - 6. * shear_modulus * r_inverse) +
            y4 * llp1 +
            y5 * density * lp1 +
            y6 * -density * radius
    )

    dy3 = \
        y1 * -r_inverse + \
        y3 * r_inverse + \
        y4 * (1. / shear_modulus)

    dy4 = r_inverse * (
            y1 * (density_gravity - 3. * two_shear_r_inv) +
            y2 * -1. +
            y3 * (two_shear_r_inv * (2. * llp1 - 1.)) +
            y4 * -3. +
            y5 * -density
    )

    dy5 = \
        y1 * grav_term + \
        y5 * -lp1 * r_inverse + \
        y6

    dy6 = r_inverse * (
            y1 * grav_term * lm1 +
            y6 * lm1 +
            y1_y3_term * grav_term
    )

    # Build output
    dy = np.empty(12, dtype=np.float64)

    # Convert back to floats
    dy[0] = np.real(dy1)
    dy[1] = np.imag(dy1)
    dy[2] = np.real(dy2)
    dy[3] = np.imag(dy2)
    dy[4] = np.real(dy3)
    dy[5] = np.imag(dy3)
    dy[6] = np.real(dy4)
    dy[7] = np.imag(dy4)
    dy[8] = np.real(dy5)
    dy[9] = np.imag(dy5)
    dy[10] = np.real(dy6)
    dy[11] = np.imag(dy6)

    return dy

In [176]:
radius = 100.
gravity = 1.5
density = 3000.
shear_modulus = 1.0e10 + 200.j
bulk_modulus = 1.0e10
frequency = 1.0e-5
G = 6.67430e-11
y = np.asarray(
        (100., 2., 100., 2., 300., 3., 400., 4., 600., 9., -33., 2.), dtype=np.float64
        )
radial_derivatives_solid_general(radius,
    y,
    shear_modulus, density,
    gravity,
    order_l = 3, G_to_use = G)

array([3.400e+01, 3.200e-01, -2.040e+10, -1.920e+08, 2.000e+00, 1.000e-02,
       1.320e+10, 1.260e+08, -5.700e+01, 1.640e+00, -6.601e-01, 4.000e-02])